In [ ]:
import sys
sys.path.append("../") 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from epimodel.preprocessing import preprocess_data, PreprocessedData
from epimodel.epiparam import EpidemiologicalParameters

In [ ]:
from epimodel.models.mixed_intervention_rw_model import mixed_intervention_rw_model

In [ ]:
import pickle

data_pickle = pickle.load(open('uk_test_set.pkl', 'rb'))

In [ ]:
data = PreprocessedData(data_pickle['regions'], data_pickle['days'], data_pickle['CMs'], data_pickle['new_cases'], data_pickle['new_deaths'], data_pickle['active_cms'])

In [ ]:
data.new_cases = np.ma.array(data.new_cases)
data.new_cases[:, :10] = np.ma.masked
data.new_deaths = np.ma.array(data.new_deaths)
data.new_deaths[:, :30] = np.ma.masked

In [ ]:
ep = EpidemiologicalParameters()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpyro
from numpyro.infer import MCMC, NUTS, Predictive
import numpyro.distributions as dist
from jax import random
import jax.numpy as jnp
import jax.scipy.signal as jss

import jax

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import time 

In [ ]:
from numpyro.infer import init_to_median
nuts_kernel = NUTS(mixed_intervention_rw_model, init_strategy = init_to_median)

mcmc = MCMC(nuts_kernel, num_samples=500, num_warmup=500, num_chains=1)
rng_key = random.PRNGKey(0)

start = time.time()

with numpyro.validation_enabled():
    res = mcmc.run(rng_key, data, ep)
    
end = time.time()

posterior_samples = mcmc.get_samples(np.array([0]))

print(f'Sampling 1000 samples per chain took {end - start}')